In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import os
import time

In [3]:
from caps_layer import *

In [4]:
class Caps_net(tf.keras.Model):

  def __init__(self,no_classes=10):
    super(Caps_net,self).__init__()
    self.no_classes=no_classes

    self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    self.loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    self.train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
    self.val_acc_metric = tf.keras.metrics.CategoricalAccuracy()

    self.pri_layer=Primary_caps_layer(caps_n=256,k1=64,k2=64,k_s1=9,k_s2=5,s1=1,s2=3)
    self.dig_layer=Digit_caps_layer(caps_dim=8,caps_n=no_classes,r=3)

    self.decoder=tf.keras.Sequential([
      keras.layers.Dense(128, activation='relu'),
      keras.layers.Dense(128, activation='relu'),
      keras.layers.Dense(256*256*3, activation='sigmoid'),
    ])

  def call(self,input_tensor,y,training=False):
    """ y should not be prob. dist/one-hot vectors it should be list of label for mnist it would 
        be as [1,4,6,3,8,7,...,5]. 
        when training is false y is not needed."""

    batch_size=input_tensor.shape[0]
    img_dim=input_tensor.shape[1] # considering image size=(img_dim,img_dim,img_depth)
    img_depth=input_tensor.shape[-1]

    x = self.pri_layer(input_tensor) #x.shape=[batch_size,caps_n(i),caps_dim(i)]
    x = self.dig_layer(x) #x.shape=[batch_size, 1,caps_n(i),caps_dim(i), 1]
    z = safe_norm(x, axis=-2) #x.shape=[batch_size,1,caps_n(i-1),1]
    z = tf.nn.softmax(z,axis=2) #converting those probabilities to prob dist.
    y_pred = tf.squeeze(z, axis=[1,3]) #reducing the extra dims. therefore the output shape =[batch_size,caps_n(i-1)] 
    if(training==False):
      return y_pred  # y_pred is a prob. dist.

    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False)(tf.one_hot(y,depth=self.no_classes), y_pred)

    #loss2 i.e reconstruction loss.
    reconstruction_mask = tf.one_hot(y,depth=self.no_classes) # recon_mask is one-hot vect rep. of y.
    
    reconstruction_mask_reshaped = tf.reshape(reconstruction_mask, [batch_size, 1, self.no_classes, 1, 1])
    # above reshape is done so that we can apply the mask.
    lastcaps_output_masked = tf.multiply(x, reconstruction_mask_reshaped)

    lastcaps_n=x.shape[2] # no of capsule in last layer.
    lastcaps_dims=x.shape[3] # dim of capsule in last layer.

    decoder_input = tf.reshape(lastcaps_output_masked,[batch_size, lastcaps_n * lastcaps_dims])
    
    decoder_output=self.decoder(decoder_input) 
    """ reconstruction of the input image based on the output vector of last layer
        we apply the mask to the output of the last layer such that only the vector corresponding to a
        particular lable is passed to the decoder."""

    X_flat = tf.reshape(input_tensor, [batch_size,img_dim*img_dim*img_depth]) 
    
    squared_difference = tf.square(X_flat - decoder_output)
    reconstruction_loss = tf.reduce_mean(squared_difference) # computation of mean squared loss between input image and reconstructed image.
  
    return loss+0.0005*reconstruction_loss

  def fit(self,train_dataset,validation_dataset,epochs=3):

    for epoch in range(epochs):
      print("\nepoch {}/{}".format(epoch+1,epochs))
      pbar = keras.utils.Progbar(target=int(train_dataset.cardinality()))
      metrics = {}

      # Iterate over the batches of the dataset.
      for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
          with tf.GradientTape() as tape:
              y_pred=self(x_batch_train,y_batch_train,training=False) # $ better design needed.
              # y_pred is prob. dist.
              loss_value = self(x_batch_train,y_batch_train,training=True) # loss computation
          grads = tape.gradient(loss_value, self.trainable_weights) # back prop
          self.optimizer.apply_gradients(zip(grads, self.trainable_weights)) # weight update

          # Update training metric.
          self.train_acc_metric.update_state(tf.keras.utils.to_categorical(y_batch_train,num_classes=self.no_classes), y_pred)
          metrics.update({'train_acc':self.train_acc_metric.result()})
          pbar.update(step+1, values=metrics.items(), finalize=False)


      # Run a validation loop at the end of each epoch.
      for x_batch_val, y_batch_val in validation_dataset:
        y_batch_val=tf.keras.utils.to_categorical(y_batch_val,num_classes=self.no_classes)
        val_pred = self(x_batch_val,y_batch_val,training=False) # $ better design needed
        # Update val metrics
        self.val_acc_metric.update_state(y_batch_val, val_pred)

      metrics.update({'val_acc':self.val_acc_metric.result()})
      
      pbar.update(step+1, values=metrics.items(), finalize=True)
      
      # Reset training & val metrics at the end of each epoch
      self.train_acc_metric.reset_states()
      self.val_acc_metric.reset_states()


In [5]:
ls drive/MyDrive/CXRDATA

Test/  Train/  Train.zip


In [6]:
PATH="drive/MyDrive/CXRDATA"

train_dir = PATH+"/Train" 
validation_dir = PATH+"/Test"

BATCH_SIZE = 32
IMG_SIZE = (256, 256)

#train data
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

print('Number of validation batches: %d' % tf.data.experimental.cardinality(train_dataset))

#validation model.
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)


# creating test data.
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))


Found 12121 files belonging to 3 classes.
Number of validation batches: 379
Found 3032 files belonging to 3 classes.
Number of validation batches: 76
Number of test batches: 19


In [7]:
#optimization parameter setting.
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [8]:
train_dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [9]:
model=Caps_net(no_classes=3)

In [10]:
model.fit(train_dataset,validation_dataset,epochs=30)


epoch 1/30
379/379 [==============================] - 1775s 5s/step - train_acc: 0.6605 - val_acc: 0.6679

epoch 2/30
379/379 [==============================] - 137s 360ms/step - train_acc: 0.6767 - val_acc: 0.6700

epoch 3/30
379/379 [==============================] - 151s 398ms/step - train_acc: 0.6749 - val_acc: 0.6716

epoch 4/30
379/379 [==============================] - 137s 360ms/step - train_acc: 0.6743 - val_acc: 0.6712

epoch 5/30
379/379 [==============================] - 151s 398ms/step - train_acc: 0.6749 - val_acc: 0.6733

epoch 6/30
379/379 [==============================] - 136s 358ms/step - train_acc: 0.6746 - val_acc: 0.6667

epoch 7/30
379/379 [==============================] - 136s 356ms/step - train_acc: 0.6763 - val_acc: 0.6712

epoch 8/30
379/379 [==============================] - 136s 357ms/step - train_acc: 0.6769 - val_acc: 0.6749

epoch 9/30
379/379 [==============================] - 151s 398ms/step - train_acc: 0.6770 - val_acc: 0.6679

epoch 10/30
379/379 

KeyboardInterrupt: ignored

In [11]:
model.save_weights('./checkpoints/my_checkpoint')